In [ ]:
from dataclasses import dataclass, field
import xml
import xmltodict
import time
import zipfile
import zlib
import brotli

from docs.test.testinnsending.altinn3 import *
from docs.test.testinnsending.skatteetaten_api import get_access_token, decode_dokument, skattemelding_visning

In [ ]:
@dataclass
class ValiderData:
    orgnr: str
    partnr: str = ""
    konvolutt: str = ""
    inntektsår: str = "2023"
    sme_b64: str = ""
    nsp_b64: str = ""
    nsp_komprimert: bytes = ""
    nsp_vedlegg_filnavn: str = ""
    sme_xml: str = ""
    nsp_xml: str = ""
    dokref_gjeldende: str = ""
    vedlegg_referanse: str = ""
    jobb_id: str = ""
    valider_data: str = ""
    jobb_status: str = ""
    resultat_av_valdiering: str = ""
    valider_resultat: dict = field(default_factory=dict)


def last_opp_vedlegg(base_url: str, valider_data: ValiderData, access_token: dict, gzip=False, bruk_brotli=False):
    api_path = f'/api/skattemelding/v2/jobb/{valider_data.inntektsår}/{valider_data.orgnr}/last-opp.vedlegg'
    url = base_url + api_path

    
    try:
        access_token.pop("Content-Type")
    except KeyError:
        pass
    
    if gzip and bruk_brotli:
        raise(NotImplementedError)

    elif gzip:
        print("Komprimerer nsp base64 vha Gzip ", end="")
        compress = zlib.compressobj(wbits=16+zlib.MAX_WBITS)
        valider_data.nsp_komprimert = compress.compress(valider_data.nsp_b64.encode("utf-8")) + compress.flush()
        valider_data.nsp_vedlegg_filnavn = valider_data.nsp_vedlegg_filnavn + ".gz"
        print("Gzip ferdig, laster opp vedlegg ", end = "")
        payload = valider_data.nsp_komprimert
    
    elif bruk_brotli:
        print("Komprimerer nsp base64 vha Brotli ", end="")
        valider_data.nsp_komprimert = brotli.compress(valider_data.nsp_b64.encode("utf-8"))
        valider_data.nsp_vedlegg_filnavn = valider_data.nsp_vedlegg_filnavn + ".br"
        print("Komprimering ved hjelp av Brotli ferdig, laster opp vedlegg ", end = "")
        payload = valider_data.nsp_komprimert
    
    else:
        payload = valider_data.nsp_b64

    files = {valider_data.nsp_vedlegg_filnavn: payload}
    return requests.post(url, files=files, headers=access_token, verify=False)


def hent_jobb(base_url: str,
              valider_data: ValiderData,
              access_token: dict,
              type: str = "status") -> requests.get:
    try:
        access_token.pop("Content-Type")
    except KeyError:
        pass

    api_path = f'/api/skattemelding/v2/jobb/{valider_data.inntektsår}/{valider_data.orgnr}/{valider_data.jobb_id}/{type}'
    url = base_url + api_path
    return requests.get(url, headers=access_token)


def hent_gjeldende(base_url: str, orgnr: str, inntekstår: int, access_token: dict):
    url = base_url + f"/api/skattemelding/v2/{inntekstår}/{orgnr}"
    return requests.get(url, headers=access_token)


def decode_gjeldende(gjeldende_response: requests.Response(), valdier_data: ValiderData):
    sme_og_naering_respons = xmltodict.parse(gjeldende_response.text)
    skattemelding_base64 = \
    sme_og_naering_respons['skattemeldingOgNaeringsspesifikasjonforespoerselResponse']['dokumenter'][
        'skattemeldingdokument']
    sme_base64 = skattemelding_base64["content"]
    dokref = sme_og_naering_respons["skattemeldingOgNaeringsspesifikasjonforespoerselResponse"]["dokumenter"][
        'skattemeldingdokument']['id']
    decoded_sme_xml = decode_dokument(skattemelding_base64)
    sme_gjeldende = xml.dom.minidom.parseString(decoded_sme_xml["content"]).toprettyxml()

    sme_dict = xmltodict.parse(decoded_sme_xml['content'])
    partsnummer = sme_dict['skattemelding']['partsnummer']

    valdier_data.sme_b64 = sme_base64
    valdier_data.sme_xml = sme_gjeldende
    valdier_data.dokref_gjeldende = dokref
    valdier_data.partnr = partsnummer
    return valdier_data


def valider_async(base_url: str,
                  valider_data: ValiderData,
                  access_token: dict) -> requests.Response:
    api_path = f"/api/skattemelding/v2/jobb/{valider_data.inntektsår}/{valider_data.orgnr}/start"
    url = base_url + api_path
    headers = dict()
    headers['Content-Type'] = "application/xml"
    headers.update(access_token)
    return requests.post(url, data=valider_data.konvolutt, headers=headers)


def lag_asynk_konvolutt(valider_data: ValiderData) -> str:
    valider_asynk_up_konvolutt = """<skattemeldingOgNaeringsspesifikasjonRequest xmlns="no:skatteetaten:fastsetting:formueinntekt:skattemeldingognaeringsspesifikasjon:request:v2">
    <dokumenter>
        <dokument>
            <type>skattemeldingUpersonlig</type>
            <encoding>utf-8</encoding>
            <content>{sme_up_base64}</content>
        </dokument>
        <dokument>
            <type>naeringsspesifikasjonReferanse</type>
            <encoding>utf-8</encoding>
            <content>{vedleggs_referanse}</content>
        </dokument>
    </dokumenter>
    <dokumentreferanseTilGjeldendeDokument>
        <dokumenttype>skattemeldingUpersonlig</dokumenttype>
        <dokumentidentifikator>{dokumentreferanse}</dokumentidentifikator>
    </dokumentreferanseTilGjeldendeDokument>
    <inntektsaar>{inntektsår}</inntektsaar>
    <innsendingsinformasjon>
        <innsendingstype>komplett</innsendingstype>
        <opprettetAv>Turboskatt</opprettetAv>
    </innsendingsinformasjon>
</skattemeldingOgNaeringsspesifikasjonRequest>"""
    return valider_asynk_up_konvolutt.format(sme_up_base64=valider_data.sme_b64,
                                             vedleggs_referanse=valider_data.vedlegg_referanse,
                                             dokumentreferanse=valider_data.dokref_gjeldende,
                                             inntektsår=valider_data.inntektsår).lstrip("\n")



In [ ]:
access_token = get_access_token()

In [ ]:
base_url = "https://idporten-api-sbstest.sits.no"
inntektsår = 2023
orgnr = "314059344"
daglig_leder = "22857799295"

valider_data = ValiderData(orgnr, inntektsår=inntektsår)
r_gjeldende = hent_gjeldende(base_url, orgnr, inntektsår, access_token)

if r_gjeldende:
    valider_data = decode_gjeldende(r_gjeldende, valider_data)

else:
    print("Obs, noe gikk galt: ")
    print(f"http {r_gjeldende.status_code}")
    print(r_gjeldende.headers)
    print(r_gjeldende.text)
    
    

In [ ]:
filnavn = "00_Mini.b64"

if filnavn.endswith("zip"):
    archive =  zipfile.ZipFile(f"../../../../src/resources/eksempler/2023/I Kraftselskap/{filnavn}")
    valider_data.nsp_b64 = file = archive.read(filnavn.replace(".zip", "")).decode("utf-8")
    valider_data.nsp_vedlegg_filnavn = filnavn

else:
    with open(f"../../../../src/resources/eksempler/2023/I Kraftselskap/{filnavn}") as f:
        valider_data.nsp_b64 = f.read()
        valider_data.nsp_vedlegg_filnavn = filnavn

In [ ]:
#Last opp NSP:
print("#Laster opp NSP: ", end="")
last_opp_vedlegg_response = last_opp_vedlegg(base_url, valider_data, access_token, gzip=True, bruk_brotli=False) #komprimerer vha gzip
valider_data.vedlegg_referanse = last_opp_vedlegg_response.json()["referanse"]
print(valider_data.vedlegg_referanse)

In [ ]:
print("#Starter valideringsjobb: ", end="")
valider_data.konvolutt = lag_asynk_konvolutt(valider_data)
valider_resultat = valider_async(base_url, valider_data, access_token)
valider_data.jobb_id = valider_resultat.json()["jobbId"]
print(valider_data.jobb_id)
if valider_data.jobb_status: # nullstiller valideringsresultat om det er kjørt ferdig før
    valider_data.jobb_status = None 

t0 = datetime.now()
print(f"Venter på at valideringsjobben {valider_data.jobb_id} skal bli ferdig, tid brukt: ", end="")
while valider_data.jobb_status not in {"FEILET", "FERDIG"}:
    time.sleep(5)
    status_response = hent_jobb(base_url, valider_data, access_token, "status")
    valider_data.jobb_status = status_response.json()["jobbStatus"]
    t1 = datetime.now() - t0
    print(f"{t1}", end="\r",)

if valider_data.jobb_status != "FEILET":
    print("\nHenter resultatet av valideringen: ", end="")
    resultat_response = hent_jobb(base_url, valider_data, access_token, "resultat")
    valider_data.valider_resultat = xmltodict.parse(resultat_response.text)
    valider_data.resultat_av_valdiering = valider_data.valider_resultat['skattemeldingOgNaeringsspesifikasjonResponse'][
        'resultatAvValidering']
    print(valider_data.resultat_av_valdiering)

else:
    print(f"Valideringsjobb med id: {valider_data.jobb_id} feilet")


In [ ]:
# Altinn 3:
altinn3_applikasjon = "skd/formueinntekt-skattemelding-v2"
altinn_header = hent_altinn_token(access_token)
tesmiljø = None 

instans_data = opprett_ny_instans_med_inntektsaar(altinn_header,
                                                  valider_data.inntektsår,
                                                  tesmiljø,
                                                  orgnr=valider_data.orgnr,
                                                  appnavn=altinn3_applikasjon)

In [ ]:
req_send_inn = last_opp_skattedata(instans_data, altinn_header,
                                   xml=valider_data.konvolutt,
                                   data_type="skattemeldingOgNaeringsspesifikasjon",
                                   appnavn=altinn3_applikasjon)

req_bekreftelse1 = endre_prosess_status(instans_data, altinn_header, "next", appnavn=altinn3_applikasjon)
req_bekreftelse2 = endre_prosess_status(instans_data, altinn_header, "next", appnavn=altinn3_applikasjon)
print(f"Lastet opp sendt inn: {instans_data["id"]}")

In [ ]:
url_skattemelding_visning = skattemelding_visning(instans_data, appnavn=altinn3_applikasjon)
print(url_skattemelding_visning)